# Global CO₂ Emissions and Energy Consumption: Trends, Forecasts, and Insights
link for dataset: https://data360.worldbank.org/en/dataset/OWID_CB

## Notebook Objective

Data Preparation (Python Notebook): Transform the raw environmental data through intensive cleaning, preprocessing, and feature engineering. The goal is to create a structured and reliable dataset that serves as the foundation for all subsequent analysis.

In [1]:
import pandas as pd
import numpy as np

In [2]:
emissions_wide = pd.read_csv('OWID_CB_WIDEF.csv')
print("Original columns:")
print(emissions_wide.columns)
emissions_wide.head()

Original columns:
Index(['FREQ', 'FREQ_LABEL', 'REF_AREA', 'REF_AREA_LABEL', 'INDICATOR',
       'INDICATOR_LABEL', 'UNIT_MEASURE', 'UNIT_MEASURE_LABEL', 'DATABASE_ID',
       'DATABASE_ID_LABEL',
       ...
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023'],
      dtype='object', length=290)


,FREQ,FREQ_LABEL,REF_AREA,REF_AREA_LABEL,INDICATOR,INDICATOR_LABEL,UNIT_MEASURE,UNIT_MEASURE_LABEL,DATABASE_ID,DATABASE_ID_LABEL,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,A,Annual,ABW,Aruba,OWID_CB_CEMENT_CO2,Annual CO2 emissions from cement - Annual emis...,T_CO2,Tonnes of CO2,OWID_CB,CO2 and Greenhouse Gas Emissions,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,A,Annual,ABW,Aruba,OWID_CB_CEMENT_CO2_PER_CAPITA,Annual CO2 emissions from cement (per capita) ...,RO,Ratio,OWID_CB,CO2 and Greenhouse Gas Emissions,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,A,Annual,ABW,Aruba,OWID_CB_CO2,Annual CO2 emissions - Annual total emissions ...,T_CO2,Tonnes of CO2,OWID_CB,CO2 and Greenhouse Gas Emissions,...,0.872,0.898,0.883,0.890,0.872,0.828,0.821,0.857,0.831,0.867
3,A,Annual,ABW,Aruba,OWID_CB_CO2_GROWTH_ABS,Annual CO2 emissions growth (abs) - Annual gro...,T_CO2,Tonnes of CO2,OWID_CB,CO2 and Greenhouse Gas Emissions,...,0.011,0.026,-0.015,0.007,-0.018,-0.044,-0.007,0.037,-0.026,0.036
4,A,Annual,ABW,Aruba,OWID_CB_CO2_GROWTH_PRCT,Annual CO2 emissions growth (%) - Annual perce...,PT,Percentage,OWID_CB,CO2 and Greenhouse Gas Emissions,...,1.277,2.941,-1.633,0.830,-2.058,-5.042,-0.885,4.464,-3.046,4.321


### Data Cleaning & Preprocessing Plan
To achieve the project objective, the following data cleaning and transformation pipeline will be implemented.

#### Step 1 Drop Useless Years
The initial dataset contains many years with  missing data. These will be removed to focus the analysis on the relevant time period.

Drop 1750–1925 (all empty).

Drop 1926–1950 (all zero).

Drop 1951–1985 (mostly Nulls and zeros).

Keep years from 1986 → latest available year.

#### Step 2 Drop Constant Metadata Columns
Several columns contain constant metadata that provides no analytical value. These will be dropped to simplify the dataset.

FREQ, FREQ_Label

DATABASE_ID, DATABASE_ID_Label

OBS_STATUS, OBS_STATUS_Label

OBS_CONF, OBS_CONF_Label

OBS_CONFIGURE, OBS_CONFIGURE_Label

In [3]:
#Step 1
years_to_drop = [str(year) for year in range(1750, 1986)]
actual_cols_to_drop_years = [col for col in emissions_wide.columns if col in years_to_drop]
emissions_wide.drop(columns=actual_cols_to_drop_years, inplace=True)

print(f"Dropped {len(actual_cols_to_drop_years)} year columns (from 1750 to 1985).")

Dropped 236 year columns (from 1750 to 1985).


In [4]:
#Step 2
metadata_cols_to_drop = [
    'FREQ', 'FREQ_Label',
    'DATABASE_ID', 'DATABASE_ID_Label',
    'OBS_STATUS', 'OBS_STATUS_Label',
    'OBS_CONF', 'OBS_CONF_Label',
    'OBS_CONFIGURE', 'OBS_CONFIGURE_Label'
]
emissions_wide.drop(columns=metadata_cols_to_drop, inplace=True, errors='ignore')

print(f"Dropped constant metadata columns.")
print("-" * 30)
print("Columns after cleaning:")
print(emissions_wide.columns)
print("\nCleaned DataFrame head:")
print(emissions_wide.head())

Dropped constant metadata columns.
------------------------------
Columns after cleaning:
Index(['FREQ_LABEL', 'REF_AREA', 'REF_AREA_LABEL', 'INDICATOR',
       'INDICATOR_LABEL', 'UNIT_MEASURE', 'UNIT_MEASURE_LABEL',
       'DATABASE_ID_LABEL', 'UNIT_MULT', 'UNIT_MULT_LABEL', 'OBS_STATUS_LABEL',
       'OBS_CONF_LABEL', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023'],
      dtype='object')

Cleaned DataFrame head:
  FREQ_LABEL REF_AREA REF_AREA_LABEL                      INDICATOR  \
0     Annual      ABW          Aruba             OWID_CB_CEMENT_CO2   
1     Annual      ABW          Aruba  OWID_CB_CEMENT_CO2_PER_CAPITA   
2     Annual      ABW          Aruba                    OWID_CB_CO2   
3     Annual 

#### Step 3 Reshape Data from Wide to Long Format
The data will be unpivoted to transform the year-based columns into rows, creating a tidy data structure suitable for analysis. The final columns will be:

Country

Indicator

Unit

Year

Value

#### Step 4 Handle Missing Values
A strategy will be implemented to handle missing data points within time series.

Drop entire groups (by Country + Indicator) that have more than 70% missing values.

Use linear interpolation to fill internal gaps within each time series group.

Use forward/backward fill to handle missing values at the beginning or end of a series.

In [5]:
#Step 3
identifier_columns = ['REF_AREA_LABEL', 'INDICATOR_LABEL', 'UNIT_MEASURE_LABEL']
year_columns = [col for col in emissions_wide.columns if col.isdigit()]
columns_to_keep = identifier_columns + year_columns
emissions_subset = emissions_wide[columns_to_keep]
emissions_long = pd.melt(emissions_subset,
                         id_vars=identifier_columns,
                         var_name='Year',
                         value_name='Value')
emissions_long['Year'] = pd.to_numeric(emissions_long['Year'])
rename_dict = {
    'REF_AREA_LABEL': 'Country',
    'INDICATOR_LABEL': 'Indicator',
    'UNIT_MEASURE_LABEL': 'Unit'
}
emissions_long.rename(columns=rename_dict, inplace=True)


print("Reshaped (long format) DataFrame head:")
print(emissions_long.head())
print("\nReshaped (long format) DataFrame tail:")
print(emissions_long.tail())
print(f"\nThe new DataFrame has {emissions_long.shape[0]} rows and {emissions_long.shape[1]} columns.")

Reshaped (long format) DataFrame head:
  Country                                          Indicator           Unit  \
0   Aruba  Annual CO2 emissions from cement - Annual emis...  Tonnes of CO2   
1   Aruba  Annual CO2 emissions from cement (per capita) ...          Ratio   
2   Aruba  Annual CO2 emissions - Annual total emissions ...  Tonnes of CO2   
3   Aruba  Annual CO2 emissions growth (abs) - Annual gro...  Tonnes of CO2   
4   Aruba  Annual CO2 emissions growth (%) - Annual perce...     Percentage   

   Year   Value  
0  1986   0.000  
1  1986   0.000  
2  1986   0.180  
3  1986  -0.757  
4  1986 -80.830  

Reshaped (long format) DataFrame tail:
         Country                                          Indicator  \
513869  Zimbabwe  Change in global mean surface temperature caus...   
513870  Zimbabwe  Total greenhouse gas emissions including land-...   
513871  Zimbabwe  Total greenhouse gas emissions excluding land-...   
513872  Zimbabwe  Annual CO2 emissions embedded in tra

In [6]:
#Step 4
print(f"Original number of rows: {len(emissions_long)}")
print(f"Original missing values in 'Value': {emissions_long['Value'].isnull().sum()}")
print("-" * 40)

missing_pct = emissions_long.groupby(['Country', 'Indicator'])['Value'].apply(lambda x: x.isnull().sum() / len(x))
groups_to_drop = missing_pct[missing_pct > 0.7].index
emissions_filtered = emissions_long[~emissions_long.set_index(['Country', 'Indicator']).index.isin(groups_to_drop)]

print(f"Rows after dropping sparse groups: {len(emissions_filtered)}")
print(f"Missing values after dropping sparse groups: {emissions_filtered['Value'].isnull().sum()}")
print("-" * 40)

emissions_filtered = emissions_filtered.sort_values(by=['Country', 'Indicator', 'Year'])

emissions_final = emissions_filtered.copy()
emissions_final['Value'] = emissions_final.groupby(['Country', 'Indicator'])['Value'].transform(
    lambda x: x.interpolate(method='linear').ffill().bfill()
)

print("Filling complete.")
print(f"Final number of rows: {len(emissions_final)}")
print(f"Final missing values in 'Value': {emissions_final['Value'].isnull().sum()}")
print("-" * 40)
print("\nFinal DataFrame head after handling missing values:")
print(emissions_final.head())

Original number of rows: 513874
Original missing values in 'Value': 27039
----------------------------------------
Rows after dropping sparse groups: 504032
Missing values after dropping sparse groups: 18027
----------------------------------------
Filling complete.
Final number of rows: 504032
Final missing values in 'Value': 0
----------------------------------------

Final DataFrame head after handling missing values:
           Country                                          Indicator   Unit  \
54     Afghanistan  Annual CO2 emissions (per capita) - Annual tot...  Ratio   
13577  Afghanistan  Annual CO2 emissions (per capita) - Annual tot...  Ratio   
27100  Afghanistan  Annual CO2 emissions (per capita) - Annual tot...  Ratio   
40623  Afghanistan  Annual CO2 emissions (per capita) - Annual tot...  Ratio   
54146  Afghanistan  Annual CO2 emissions (per capita) - Annual tot...  Ratio   

       Year  Value  
54     1986  0.274  
13577  1987  0.273  
27100  1988  0.248  
40623  198

#### Step 5: Clean and Standardize Categorical Columns
This step refines the primary text-based columns (Indicator and Unit) to improve their usability for analysis.

Clean Indicator Names: The verbose Indicator names will be shortened to their core descriptions (e.g., "Annual CO2 emissions - Annual..." becomes "Annual CO2 emissions"). This enhances readability and simplifies filtering.

Standardize Units: The varied strings in the Unit column will be mapped to a new, standardized Unit_Category column (e.g., 'Mass', 'Energy', 'Ratio'). This creates a consistent, high-level feature that allows for broad filtering and aggregation.

In [7]:
#Step 5
emissions_final['Indicator'] = emissions_final['Indicator'].str.split(' - ').str[0]

print("DataFrame after cleaning Indicator names:")
print(emissions_final.head())

unit_mapping = {
    'Tonnes of CO2': 'Mass',
    'Tonnes of CO2-equivalent': 'Mass',
    'Kilograms': 'Mass',
    'Ratio': 'Ratio',
    'Percentage': 'Percentage',
    'Number': 'Count',
    'Persons': 'Count',
    'Degrees celsius (Â°C)': 'Temperature',
    'PPP dollars, 2011 constant prices': 'Financial',
    'Terawatt-hours per year': 'Energy',
    'Kilowatt hour (KWh) per year': 'Energy',
    'Kilowatt hour (KWh) per PPP': 'Energy Efficiency'
}
emissions_final['Unit_Category'] = emissions_final['Unit'].map(unit_mapping)

print("DataFrame with new 'Unit_Category' column:")
print(emissions_final[['Indicator', 'Unit', 'Unit_Category']].head())
print("\nCheck for unmapped units:")
print(emissions_final['Unit_Category'].isnull().sum())

DataFrame after cleaning Indicator names:
           Country                          Indicator   Unit  Year  Value
54     Afghanistan  Annual CO2 emissions (per capita)  Ratio  1986  0.274
13577  Afghanistan  Annual CO2 emissions (per capita)  Ratio  1987  0.273
27100  Afghanistan  Annual CO2 emissions (per capita)  Ratio  1988  0.248
40623  Afghanistan  Annual CO2 emissions (per capita)  Ratio  1989  0.233
54146  Afghanistan  Annual CO2 emissions (per capita)  Ratio  1990  0.168
DataFrame with new 'Unit_Category' column:
                               Indicator   Unit Unit_Category
54     Annual CO2 emissions (per capita)  Ratio         Ratio
13577  Annual CO2 emissions (per capita)  Ratio         Ratio
27100  Annual CO2 emissions (per capita)  Ratio         Ratio
40623  Annual CO2 emissions (per capita)  Ratio         Ratio
54146  Annual CO2 emissions (per capita)  Ratio         Ratio

Check for unmapped units:
0


## Next Steps: Transition to Power BI
The data preparation phase in Python is now complete. All steps, including data cleaning, reshaping, handling missing values, and feature engineering, have been successfully executed.

The final, clean dataset has been exported to emissions_cleaned.csv.

The project will now move to Microsoft Power BI for the analysis and visualization phase. The objective is to build an interactive dashboard to explore global emissions trends, compare country-level data, and uncover key insights from the dataset.

In [8]:
emissions_final.to_csv('emissions_cleaned.csv', index=False)
print("DataFrame successfully exported to 'emissions_cleaned.csv'")

DataFrame successfully exported to 'emissions_cleaned.csv'
